In [3]:
from flask import Flask, request, jsonify
import requests

app = Flask(__name__)

FAKE_STORE_API_URL = "https://fakestoreapi.com/products"

@app.route('/chat', methods=['POST'])
def chat():
    user_message = request.json.get('message')
    response = handle_message(user_message)
    return jsonify({"response": response})

def handle_message(message):
    if 'buy' in message.lower() or 'purchase' in message.lower():
        product_name = extract_product_name(message)
        product_info = search_product(product_name)
        if product_info:
            return f"Here is the information for {product_name}: {product_info}"
        else:
            return f"Sorry, I couldn't find any information for {product_name}."
    else:
        return "How can I help you today?"

def extract_product_name(message):
    # Simple extraction for demonstration; can be improved with NLP techniques
    words = message.split()
    for i, word in enumerate(words):
        if word.lower() in ['buy', 'purchase']:
            return ' '.join(words[i+1:])
    return ""

def search_product(product_name):
    response = requests.get(FAKE_STORE_API_URL)
    if response.status_code == 200:
        products = response.json()
        for product in products:
            if product_name.lower() in product['title'].lower():
                return f"Title: {product['title']}, Price: ${product['price']}, Description: {product['description']}"
    return None

if __name__ == '__main__':
    app.run(debug=True,port=5001)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
 * Restarting with stat


Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/mnt/c/Users/vijet/twinciti-vijeth/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/mnt/c/Users/vijet/twinciti-vijeth/lib/python3.10/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/mnt/c/Users/vijet/twinciti-vijeth/lib/python3.10/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, *args, **kwargs)
  File "/mnt/c/Users/vijet/twinciti-vijeth/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 692, in initialize
    self.init_sockets()
  File "/mnt/c/Users/vijet/twinciti-vijeth/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 331, in init_sockets
    self.shell_p

SystemExit: 1

In [ ]:
from flask import Flask, request, jsonify, render_template, session, make_response
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from groq import Groq
import time
from webdriver_manager.chrome import ChromeDriverManager
from selenium_stealth import stealth
import traceback
import threading
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from groq import Groq
import time
from webdriver_manager.chrome import ChromeDriverManager
import traceback
from selenium_stealth import stealth
import argparse
import os
import time
from threading import Thread, Lock
from queue import Queue
from dotenv import load_dotenv
from tqdm import tqdm
import pymongo
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bson import ObjectId
from datetime import datetime
from langdetect import detect
import re
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed
import queue
import signal
import psutil
import concurrent.futures
from selenium.common.exceptions import NoSuchElementException
from dateutil.parser import parse
import traceback
from selenium.common.exceptions import TimeoutException

app = Flask(__name__)
#app.secret_key = "123123123"  # Use a default value if the environment variable is not set

def load_driver():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("start-maximized")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument("--log-level=3")
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108 Safari/537.3")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])

    try:
        service = Service(ChromeDriverManager().install())  
        driver = webdriver.Chrome(service=service, options=chrome_options)
        stealth(driver,
            languages=["en-US", "en"],
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine",
            fix_hairline=True,
            )
        return driver
    except Exception as e:
        traceback.print_exc()
        raise
class ShoppingAssistant:
    def __init__(self, llm_client):
        self.llm_client = llm_client
        self.driver = load_driver()
        self.lock = threading.Lock()

    def get_product_info(self, product_name):
        url = "https://www.amazon.in/s?k=" + product_name
        #print(url)

        self.driver.get(url)
        time.sleep(5)

        products = self.driver.find_elements(By.CSS_SELECTOR, "span.a-size-medium.a-color-base.a-text-normal")
        return [product.text for product in products[:5]]

    def get_ai_response(self, messages):
        #print(messages)
        completion = self.llm_client.chat.completions.create(
            model="llama3-8b-8192",
            messages=messages,
            temperature=1,
            max_tokens=1024,
            top_p=1,
            stream=True,
            stop=None,
        )
        response = ""
        for chunk in completion:
            response += chunk.choices[0].delta.content or ""
        return response.strip()

    def handle_message(self, message):
        

        session.append({"role": "user", "content": message})

        # First classification: Is the prompt related to buying a product?
        classification_messages = [
            {"role": "system", "content": "Answer in yes or no. You are an amazon product assistant. Is the below prompt related to buying a product or could it lead to this?"},
            {"role": "user", "content": message}
        ]
        classification_response = self.get_ai_response(classification_messages)

        if "yes" in classification_response.lower():
            # Second classification: Can the prompt guide a search on the website?
            classification_messages = [
                {"role": "system", "content": "Given that you are a search tool in a shopping website, does the below prompt contain any information that can guide you to search on the website searchbar? Answer only in yes or no"},
                {"role": "user", "content": message}
            ]
            classification_response = self.get_ai_response(classification_messages)

            if "yes" in classification_response.lower():
                # Extract search query
                classification_messages = [
                    {"role": "system", "content": "Given that you are a search tool in a shopping website, does the below prompt contain any information that can guide you to search on the website searchbar? Answer only what you will search in the searchbar"},
                    {"role": "user", "content": message}
                ]
                product_name = self.get_ai_response(classification_messages)
                product_name = ''.join([char for char in product_name if char.isalpha() or char == '+'])

                # Fetch matching products
                matching_products = self.get_product_info(product_name)

                if matching_products:
                    response_content = f"Top 5 matching product titles:<br><br>" + "<br><br>".join(matching_products)
                    session.append({"role": "assistant", "content": response_content})

                    # ask user if he wants the ai to select the best product
                    # show two buttons in the interface yes and no to be clicked by the user
                    # if user clicks yes. then create a prompt with sending matching_products to ai llm and ask it to choose the best and provide reason

                    
                else:
                    response_content = "Sorry, I couldn't find any matching products on Amazon."
            else:
                response_content = "Please provide more information on the product you want to purchase."
        else:
            # Non-product related response
            ai_response = self.get_ai_response(session)
            session.append({"role": "assistant", "content": ai_response})
            response_content = ai_response

        # Store the AI's response in the session
        #session['conversation'].append({"role": "assistant", "content": response_content})
        session = session[-15:]
        return response_content


    def __del__(self):
        self.driver.quit()


llm_client = Groq(api_key="")
assistant = ShoppingAssistant(llm_client)

session = []

@app.before_request
def clear_cookie_on_refresh():
    if request.endpoint == 'index':  # Only clear session for the index route (page refresh)
        session = [] # Clear the session

@app.route('/')
def index():
    
    return render_template('index.html')

@app.route('/chat', methods=['POST'])
def chat():
    data = request.json
    user_message = data.get("message")
    if not user_message:
        return jsonify({"error": "No message provided"}), 400
    response = assistant.handle_message(user_message)
    return jsonify({"response": response})

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0', port=5000)
